# Nucleo TNC

The Nucleo TNC is a breadboard implementation of the Mobilinkd TNC3 using a STM32L432KC Nucleo32 board.  This TNC faithfully implements the audio section and EEPROM storage of the TNC3.  It omits the battery charging and Bluetooth components of the TNC3.  It presents as a KISS TNC over a USB serial port.

The Nucleo TNC can be assembled for about $25 in parts, assuming you already have a breadboard and hookup wire.  The bill of materials and the equipment needed to build and use the TNC are provided below in the *Bill of Materials* and *Equipment* sections, respectively.

This project is intended for amateur radio operators from beginners to advanced.  If you want to just dive in to building it without understanding the theory of operations, start at the overview and skip to the BOM and Assembly Instructions.  For those who wish to understand the theory of operation from an electrical perspective, we start with the *Schematic* and *Theory of Operation* sections after a brief overview.

And for those who wish to understand the digital signal processing involved in this project, please refer to the [AFSK Demodulator](https://github.com/mobilinkd/afsk-demodulator/blob/master/afsk-demodulator.ipynb) tutorial which outlines the basics of how the demodulator is constructed.  You can also refer to the source repository for the firmware.

This breadboard TNC has the same radio interface pinout as the Mobilinkd TNCs and is designed to be used with the same cables as a Mobilinkd TNC.  Pre-made cables for your radio can be purchased at http://store.mobilinkd.com or you can make your own.

![Step 10](AssemblyStep10.png)

## Table of Contents

 * [Schematic](#Schematic)
 * [Theory of Operation](#Theory-of-Operation)
   * [Radio Connection](#Radio-Connection)
   * [PTT Circuitry](#PTT-Circuitry)
   * [EEPROM](#EEPROM)
   * [Indicator LEDs](#Indicator-LEDs)
   * [Audio Input](#Audio-Input)
   * [Audio Output](#Audio-Output)
   * [Nucleo Board](#Nucleo-Board)
 * [Bill of Materials](#Bill-of-Materials)
 * [Equipment](#Equipment)
 * [Assembly](#Assembly)
   * [Nucleo and ICs](#Nucleo-and-ICs)
   * [Bypass Capacitors](#Bypass-Capacitors)
   * [Power and Ground](#Power-and-Ground)
   * [LEDs](#LEDs)
   * [3.5mm Jack](#3.5mm-Jack)
   * [PTT](#PTT)
   * [EEPROM Connections](#EEPROM-Connections)
   * [Audio Input Circuitry](#Audio-Input-Circuitry)
   * [Audio Output Circuitry](#Audio-Output-Circuitry)
   * [Wiring](#Wiring)
 * [Programming](#Programming)
 * [Operation](#Operation)
 * [Troubleshooting and Testing](#Troubleshooting-and-Testing)
 
## Overview

The Nucleo TNC is part of a larger system for sending and receiving digital information over the radio. This TNC is designed for 1200 baud AFSK, which means it is designed to send data over an audio channel -- specifically over a standard FM voice channel.  One of the more popular uses for this is for APRS.  It can also be used for Winlink, or to attach to packet radio BBS systems, and for direct one-on-one communication.

To use this TNC you will need the following:

 * A radio -- typically a hand-held radio or mobile radio.
 * The TNC itself.
 * A computer with a USB port or a mobile device (Android, iOS) with a USB-OTG interface.
  
```
    RADIO <-> Nucleo TNC <-> COMPUTER
```

 * The radio is responsible for sending and receiving the modulated data signals over the air.
 * The TNC is responsible for decoding the data received byt the radio into packets and sending them to the computer; and for receiving packets from the computer, modulating the signal, and causing the radio to transmit the data over the air.
 * The computer is responsible dispaying the data in a human-friendly way.

Because the radio is an integral part of this system, the quality of the radio plays a big part in the quality of the system as a whole.  The TNC outlined here is a high-performane TNC that is rendered mediocre with a cheap radio.

And because the computer is an integral part of the system, the software available on the computer and operating systems of choice dictate what packet radio options are available.  For example, as of the time this was written, there is no Winlink software that will work with a KISS TNC on MacOS or Android.

Also, because Apple restrict USB access via iOS, just as it does with Bluetooth SPP, you cannot access the TNC from an iOS device at this time.  Apple users should complain loudly about this.  It's an asinine policy.

This document is divided into a number of sections.  We start with the schematic diagram and theory of operation, then go into the bill of materials, the tools you will need, construction, troubleshooting, and operation.

## References

 * [MBed NUCLEO-L432KC](https://os.mbed.com/platforms/ST-Nucleo-L432KC/)
 * [MCP 6004 Datasheet](https://ww1.microchip.com/downloads/en/DeviceDoc/21733j.pdf)
 * [MCP 24LC32A Datasheet](http://ww1.microchip.com/downloads/en/devicedoc/21713m.pdf)
 * [2N7000 Datasheet](http://ww1.microchip.com/downloads/en/DeviceDoc/2N7000.pdf)

## Glossary

ADC - analog to digital converter. An electronic component that converts analog voltages to digital numbers.
APRS - Automatic Packet Reporting System. A packet radio system invented by Bob Bruninga, WB4APR.
BBS - Bulletin Board System. An online interactive computer system common before the advent of the Internet.
DAC - digital to analog converter. An electronic component that converts digital numbers to analog voltages.
EEPROM - electrically-erasable programmable read-only memory.  An integrated circuit for semi-permanent data storage.
I2C - a two-wire digital communications bus used by integrated circuits for inter-chip communication.  It contains a clock line (SCL) and a data line (SDA). https://en.wikipedia.org/wiki/I%C2%B2C
KISS - Acronym for *Keep it Simple, Stupid*.  A type of TNC defined by a minimal control interface.  This is in contrast to other modems that have a full human-readable command language.
LED - light emitting diode.
MCU - microcontroller unit. A microprocessor with embedded memory and peripherals.
PGA - programmable gain amplifier.  An amplifier whose gain can be set programmatically.
PTT - Push-to-talk. The mechanism for enabling the transmitter on a radio.
TNC - terminal node controller.  In our case it refers to a modulator/demodulator (a modem).
TRRS - Tip, ring, ring, sleeve. Short-hand for a 4-pole phone connector.


## Schematic

This schematic is an SVG file so it should scale up infinitely.

![Nucleo32 TNC Schematic](schematic.svg)

The schematic consists of a number of block:

 * Audio Connector (audio in, audio out, PTT control)
 * PTT circuitry for both simplex PTT and multiplex PTT.
 * EEPROM storage for TNC configuration data.
 * LEDs for TX/RX indication.
 * Audio input processing.
 * Audio output processing.
 * The STM32 microcontroller board.
 
## Theory of Operation

In this section we will discuss all of the section at least briefly, but will focus most of the attention on the audio input and output sections and the PTT circuitry.

The audio input and output circuitry share a single quad op-amp IC.

The microcontroller (MCU) used in this project is an STM32L432KC on an ST Microelectronics Nucleo-32 board. This has a similar size an pin compatibility with the Arduino Nano.  It contains a Cortex-M4F CPU capable of running at up to 80MHz, a 12-bit analog to digital converter (ADC) capable of over-sampling to achieve up to 16-bit resolution, and a dual-channel 12-bit digital to analog converter (DAC).  It also includes a programmable gain amplifier (PGA) with up to 16X (24dB) amplification.  The MCU has 64KB of static RAM and 256KB of flash program memory.  It is an order of magnitude more capable then the Aduino Nano used in the previous breadboard TNC project.

Both the EEPROM and the quad op amp have 100nF decoupling caps across VDD and GND.

### Radio Connection

The TNC radio interface consists of a 4-pole (TRRS) 3.5mm phone jack, similar to what is used in most mobile phones (for those that still have headphone jacks).  The 4 poles are referred to as Tip (T), Ring 1 (R1), Ring 2 (R2) and Sleeve (S).  Ring 1 is closest to the tip and ring 2 is closest to the base or sleeve.

The connector pinout is:

 * Tip - TNC audio input / radio speaker.
 * Ring 1 - PTT signal for simplex PTT.  This is used by Kenwood radios and MiniDIN-6 packet interfaces.
 * Ring 2 - TNC audio output / radio mic+ input, as well as multiplex PTT for Yaesu, Icom, Alinco (and other) radios.
 * Sleeve - Common ground.

We will discuss the PTT circuitry and options affecting Ring 1 and Ring 2 in the next section.

### PTT Circuitry

There are two types of PTT systems in common use on amateur radios: simplex and multiplex.

Simplex is used on Kenwood HTs, as well as all HTs that use the Kenwood connector.  This includes most of the cheap Chinese radios flooding the western markets.  It is also the dominant (only?) PTT signaling system used on mobile and base station radios.  It is the PTT signalling mechanism used on the standard MiniDIN-6 packet radio interface.

Multiplex signalling is used on many hand-held radios and was the dominant signalling method on HTs for a while -- until the Chinese radios standardized on the Kenwood interface.  Most Yaesu, Icom, Alinco and other brands still use this method.


#### Simplex PTT

Simplex PTT is rather straight-forward.  A dedicated PTT signal line is held high by a weak pull-up on the radio. When this signal is brought low by connecting it to ground, the transmitter is engaged.  On the TNC, this signal line is connected to ground using a N-channel MOSFET.  When the gate on the MOSFET goes high, the PTT signal is connected to ground through the MOSFET, engaging the transmitter.  The MOSFET gate is pulled low (normally off) with a 100K pull-down resistor.

There are three components to this circuit:

 * Q2 - this is a jelly-bean N-Channel MOSFET
 * R13 - gate resistor
 * R11 - pull-down resistor

The signal from the MCU pin PA12 is active HIGH.  This turns PTT on and off.  The 2N7000 MOSFET connects PTT directly to ground when active.  The 220 Ohm gate resistor R13 is there to prevent gate ringing on the MOSFET and the 100K pull-down resistor R11 is to ensure the PTT connection is OFF even when the MCU pin is not driven.


#### Mulitplex PTT

Multiplex PTT relies on applying a DC bias to the radio's MIC+ signal by connecting it to ground through a 2.2K resistor.  The bias is held high by the radio via a weak pull-up resistor internal to the radio.  When the MIC+ line is connected to a 2.2K resistor to ground, this shifts the DC bias down and triggers PTT.  Similar to the simplex PTT circuitry, the multiplex PTT circuit uses an N-channel MOSFET to switch this PTT signal on and off.

There are four components to this circuit:

 * Q1 - this is a jelly-bean N-Channel MOSFET
 * R4 - PTT bias resistor
 * R6 - gate resistor
 * R5 - pull-down resistor

The signal from the MCU  pin PA11 is active HIGH.  This turns PTT on and off.  The 2N7000 MOSFET connects MIC+ to ground through R4 when active to bias the MIC+ line.  Just like the simples PTT circuit, the 220 Ohm gate resistor R6 is there to prevent gate ringing on the MOSFET and the 100K pull-down resistor R5 is to ensure the PTT connection is OFF even when the MCU pin is not driven.

### EEPROM

The TNC requires storage for configuration information (PTT style, KISS parameters, audio twist, etc). The STM32L4 series does not contain an internal EEPROM so we rely on an external EEPROM for this.  The EEPROM uses a 400kHz I2C interface.

The EEPROM circuitry consists of 4 components:

 * U2 - a 24LC32 32-kbit (4096 byte) EEPROM using an I2C interface
 * C10 - a 100nF bypass capacitor
 * R17 and R18 - two 2.2K pull-up resistors on the I2C lines, SDA and SCL

The SCL and SDA lines of the EEPROM (U2) are connected to PA7 and PB4 on the MCU respectively.  I2C requires external 2.2K pullups (R17 & R18) on these lines.

The power pins on the EEPROMS are bypassed using a 100nF capacitor (C10).

The EEPROM has 3 address pins and a write protect (WP) pins.  We tie all of the address pins to GND to ensure the EEPROM uses the base address value.  The WP pin is tied to GND to ensure that the EEPROM can be written.  We do not need to write protect the EEPROM for this system.

### Indicator LEDs

There are two indicator LEDs.  These are used to indicate transmit, when PTT is enabled, and receive, for data carrier detect (DCD).

The DCD LED (D3) is connected to 3.3V via a 470 Ohm current limiting resistor (R19) to its anode, and then to pin PA9 on the MCU, which is configured in open drain mode.  The LED_DCD MCU pin is active low.

Likewise, the TX LED (D4) is connected to 3.3V via a 470 Ohm current limiting resistor (R20) to its anode, and then to pin PA8 on the MCU, which is configured in open drain mode.  The LED_TX MCU pin is active low.

### Audio Input

The audio input has a nominal peak-to-peak voltage limit of 3.3V.  To achieve good decode performance, it needs a nominal 200mV peak-to-peak input level.  However it can work with input levels much lower than this.

The audio input circuit starts with a DC-blocking capacitor and a programmable DC-offset section required to set the proper DC bias for the audio section. This programmable DC offset is required because there is an additional component of the audio input circuit that is not apparent in the schematic.  There is a built-in programmable gain amplifier (PGA) in the MCU that is used and this requires varying DC offsets depending on the gain selected.  The gain is selectable from 0dB, 6dB, 12dB, 18db, & 24dB.

The input voltage is limited via two Schottkey diodes and then fed into a buffer.  The audio is fed through a 3-pole Bessel anti-aliasing filter with a cut-off of 7200Hz.  This is then fed into the PGA and ADC of the MCU.

There are a number of components in this subsection:

 * C2 - 220nF DC blocking capacitor.
 * R1 - 10K DC-offset resistor, along with C2 forms a 72Hz high-pass filter.
 * D1, D2 - voltage limiter.
 * U1C - audio buffer.
 * R7, R10, R14, C4, C6, C8, and U1D - 3-pole active Bessel anti-aliasing filter.
 
The quad op amp U1 is shared across the Audio Input and Audio Output sections.
 
Audio signal from the radio is fed through the capacitor C2 to block DC from the radio.  A DC bias voltage fed in from the MCU's DAC channel 2 (PA5) through a 10K resistor (R1), forming a high-pass filter.  This then passes through a pair of diodes which limit the voltage begin fed into the op-amp.  Amplified audio from a radio's speaker output can easily exceed the voltage limits of the op amp inputs. The op-amp U1C is an audio buffer to provide the low output impedance to feed the anti-aliasing filter.

The anti-aliasing filter is a low-pass filter designed to limit noise beyond the Nyquist range of the analog to digital converter (ADC) in the microcontroller.  The ADC samples the audio data at 26400 samples per second.  The Nyquist frequency is half the sample rate, or 13.2kHz.   Any noise in the input signal beyond the Nyquist range will be folded back into the sampled data.  The anti-aliasing filter limits the power of the noise outside the Nyquist range by at least 20dB.

https://en.wikipedia.org/wiki/Anti-aliasing_filter

After the anti-alaising filter, the audio enters the MCU.  Inside the MCU is a programmable gain amplifier (PGA) which can amplify the audio signal in discrete steps of 1X (0dB), 2X, 4X, 8X, and 16X.  The DC offset of the amplifier changes based on the amplification level from 1/2 to 1/32 (1.65V - 100mV).  The bias voltage output by DAC channel 2 is adjusted in step with the amplification level by the firmware in the TNC.

### Audio Output

Audio output from the TNC is generated by the MCU on DAC channel 1 (PA4).  This is output at 26400 samples per second, resulting in 22 samples per bit.  The output of the DAC looks like stair-steps, with discrete jumps in the various levels.

The output of the DAC is fed into a low-pass reconstruction filter, which is a 7200Hz, 3-pole Bessel filter, exactly the same as the anti-aliasing filter on the input side.  The output of the reconstruction filter results in a smooth, sine-wave output expected from an audio signal.

The outut of the DAC has about 24dB of dynamic range for output volume control.  Immediately after the reconstruction filter there is a 1/16 voltage divider formed by R8 and R9 resulting in a 24dB attenuation when PB5 is pulled low on the MCU.  This divider give the output stage a range of about 48dB.

The output of filter is then buffered and passes through a decoupling cap C2 to the MIC+ line of the radio.  The resistors R2 and R3 are there to ensure stability of the output stage.  C3 ensures proper isolation from GND.  I cannot find my notes on exactly why these were added.

The audio output section has the following components:

 * R12, R15, R16, C5, C9, C7, U1A - 3-pole Bessel reconstruction filter.
 * R8, R9 - switchable 24dB audio attenuator.
 * U1B - buffer.
 * C2, C3, R2, R3 - AC coupling capacitor and output compensation.


### Nucleo Board

The TNC uses an STM32L432KC Nucleo32 board.  This contains a 32-bit ARM Cortex-M4F CPU, along with numerous peripherals, 64kB of SRAM and 256KB or flash program memory.

https://os.mbed.com/platforms/ST-Nucleo-L432KC/

The Nucleo board presents two interfaces when connected to a computer: a serial interface and a mass storage device.  The mass storage device is used to program the MCU.  Firmware is copied to the mass storage device and instantly uploaded to the microcontroller.  The serial interface connects to pins PA2 (TX) and PA15 (RX).  These two pins are not exposed on the Nucleo board.

The Nucleo board provides power to the breadboard via the +3V3 pin and the two GND pins.  The TNC3 was designed for very low power. The circuitry we will be implementing draws very little additional power from the board.

The microcontroller can run at up to 80MHz.  For this project we will run the MCU at 48MHz, which is still more than we really need.

## Bill of Materials

To build this project, you will need the following items:

 * Standard 830 point breadboard (such as an Elenco Model 9425)
 * Jumper wire.  I prefer pre-made flexible wires with pin headers, along with short stuff jumpers for connecting power rails.
 * Micro-USB cable to connect the computer to the Nucleo board for both programming and accessing the TNC.
 
Breadboard TNC Components:

 
| Manufacturer                         	| Manufacturer Part Number 	| Description                      	| Quantity 	| Unit Price 	| Extended Price 	|
|--------------------------------------	|--------------------------	|----------------------------------	|----------	|------------	|----------------	|
| SparkFun Electronics                 	| BOB-11570                	| SPARKFUN TRRS 3.5MM JACK BREAKOU 	| 1        	| 3.95       	| 3.95           	|
| Vishay Semiconductor Diodes Division 	| BAT85S-TAP               	| DIODE SCHOTTKY 30V 200MA DO35    	| 2        	| 0.38       	| 0.76           	|
| STMicroelectronics                   	| NUCLEO-L432KC            	| NUCLEO-32 STM32L432KC EVAL BRD   	| 1        	| 10.99      	| 10.99          	|
| Microchip Technology                 	| MCP6004-I/P              	| IC OPAMP GP 4 CIRCUIT 14DIP      	| 1        	| 0.45       	| 0.45           	|
| Microchip Technology                 	| 24LC32A-I/P              	| IC EEPROM 32K I2C 400KHZ 8DIP    	| 1        	| 0.41       	| 0.41           	|
| Microchip Technology                 	| 2N7000-G                 	| MOSFET N-CH 60V 0.2A TO92-3      	| 2        	| 0.38       	| 0.76           	|
| Broadcom Limited                     	| HLMP-1301                	| LED RED DIFFUSED T-1 T/H         	| 1        	| 0.41       	| 0.41           	|
| Broadcom Limited                     	| HLMP-1503                	| LED GREEN DIFFUSED T-1 T/H       	| 1        	| 0.41       	| 0.41           	|
| Stackpole Electronics Inc            	| RNF18FTD100K             	| RES 100K OHM 1/8W 1% AXIAL       	| 3        	| 0.1        	| 0.30           	|
| Stackpole Electronics Inc            	| RNF14FTD10K0             	| RES 10K OHM 1/4W 1% AXIAL        	| 1        	| 0.1        	| 0.10           	|
| Stackpole Electronics Inc            	| RNMF14FTC510R            	| RES 510 OHM 1/4W 1% AXIAL        	| 2        	| 0.1        	| 0.20           	|
| Stackpole Electronics Inc            	| RNMF14FTC6K80            	| RES 6.8K OHM 1/4W 1% AXIAL       	| 1        	| 0.1        	| 0.10           	|
| Stackpole Electronics Inc            	| RNF14FTD470R             	| RES 470 OHM 1/4W 1% AXIAL        	| 4        	| 0.1        	| 0.40           	|
| Stackpole Electronics Inc            	| RNMF14FTC220R            	| RES 220 OHM 1/4W 1% AXIAL        	| 2        	| 0.1        	| 0.20           	|
| Stackpole Electronics Inc            	| RNMF14FTC2K20            	| RES 2.2K OHM 1/4W 1% AXIAL       	| 3        	| 0.1        	| 0.30           	|
| Stackpole Electronics Inc            	| RNMF14FTC16K0            	| RES 16K OHM 1/4W 1% AXIAL        	| 2        	| 0.1        	| 0.20           	|
| Stackpole Electronics Inc            	| RNMF14FTC30K0            	| RES 30K OHM 1/4W 1% AXIAL        	| 2        	| 0.1        	| 0.20           	|
| Vishay BC Components                 	| K471J15C0GF53L2          	| CAP CER 470PF 50V C0G/NP0 RADIAL 	| 2        	| 0.22       	| 0.44           	|
| AVX Corporation                      	| SR201A102JAR             	| CAP CER 1000PF 100V C0G/NP0 RAD  	| 2        	| 0.29       	| 0.58           	|
| Vishay BC Components                 	| K103M15X7RF53L2          	| CAP CER 10000PF 50V X7R RADIAL   	| 1        	| 0.21       	| 0.21           	|
| AVX Corporation                      	| SR215C104KAA             	| CAP CER 0.1UF 50V X7R RADIAL     	| 2        	| 0.2        	| 0.40           	|
| TDK Corporation                      	| FG14X7R1H224KNT00        	| CAP CER 0.22UF 50V X7R RADIAL    	| 1        	| 0.3        	| 0.30           	|
| TDK Corporation                      	| FG28X7R1A225KRT06        	| CAP CER 2.2UF 10V X7R RADIAL     	| 1        	| 0.34       	| 0.34           	|
| Vishay BC Components                 	| K333K15X7RF5TL2        	| CAP CER 0.033UF 50V X7R RADIAL   	| 2        	| 0.23        	| 0.46           	|


The total component cost for these items is about $25 USD from Digikey.

You can purchase these directly from Digi-Key using this link: https://www.digikey.com/short/phmpd4

It might be worthwhile to buy an extra one of each of the passive components in case one is lost or a lead is accidentally cut too short.


### Substitutions

Note that we are using high-quality C0G/NP0 capacitors and 1% resistors in the audio filters in order to ensure the best accuracy.  This is what we use in the TNC3.  It is certainly possible to substitute less expensive components if you have them readily available.  We specify 3mm T-1 LEDs.  You can certainly use 5mm LEDS if that's what you have.  You can replace the MOSFETs with general purpose N-type transistors.  You can also substitute two dual-channel op amps for the one quad op amp.  Just ensure that your op amp has a similar 1MHz bandwidth.  This is required by the active filter.  And you will need to address the layout changes required as a result of any part change.

The actual resistor and capacitor values used by the audio section are important.

The size of the EEPROM cannot be changed because doing so changes the page size, which is fixed in the firmware.

The Nucleo board cannot be substituted.  *The firmware will only work with the specified board.*


## Equipment

In order to build and test the Nucleo TNC, you will need:

 * An amateur radio license.
 * A 2-meter VHF amateur radio.
 * A computer running Linux.
 * Either a home-made cable to connect the radio to the TNC or a cable from https://store.mobilinkd.com/collections/tnc-cables
 * A dummy load and a cable to connect the radio to the dummy load.
 * An external antenna, that can be mounted well away from the breadboard.
 * Diagonal cutters to trim the leads of the passive components (resistors, capacitors, and diodes).
 * Grounded anti-static mat and wrist-strap.
 * Access to USB power from the assembly area.
 
A dummy load allows an amateur radio operator to test his or her equipment without interfering with others, and without accidentally damaging their own equipment.  It is an important tool in every experimenter's toolbox.

Nice to have equipment:

 * An extra 2M radio.
 * An RTL-SDR (Ultra-cheap SDR).
 * A digital multimeter (DMM).
 * An oscilloscope.
 * A signal generator.


## Assembly

The assembly process is divided into a few different steps, most of them following the major components in the schematic.

 * Install the larger components: Nucleo board, op amp, EEPROM, phone jack.
 * Install the bypass capacitors for the op amp and EEPROM.
 * Connect power and ground rails.
 * Install the LEDs and their current limiting resistors.
 * Install the PTT circuitry: MOSFETS & resistors.
 * Install the I2C pull-up resistors for the EEPROM.
 * Install the diodes, resistors and capacitors for the audio input section.
 * Install the resistors and capacitors for the audio input section.
 * Install the resistors and capacitors for the audio output section.
 * Connect the Nucleo board to the various subsystems.
 * Connect the rest of the components together.
 
Breaking the assembly down in this manner will allow us to test the various subsystems as we go to ensure they are working correctly.

These assembly instructions assume you can plug the Nucleo board into USB power for testing.

Note that the breadboard is labelled with rows 1-63 and columns a-j.  We will be using these to align the parts to the board.

<DIV style="background-color: #ffffce">
Assemble the Nucleo TNC on a well-grouned anti-static mat and wear a grounding wrist strap when assembling the board.  The Nucleo board, the ICs and the MOSFETs are static-sensitive devices and can be damaged by static discharge from your body that you cannot feel.
</DIV>


### Nucleo and ICs

**Parts Required:**

 * Nucleo board
 * MCP6004 quad op amp IC
 * 24LC32A EEPROM IC.

Install the Nucleo board, EEPROM and quad op-amp as shown.  Note the orientation of the components.

 * The Nucleo board starts at row 1. Insert the board pins into columns *d* and *h*.  Orient the board so that the USB port is nearest row 1, so that it is easily accessible.
 * Insert the 24LC32 EEPROM into rows 21-24, columns *e* and *f*, with pin 1 on row 21.
 * Insert the MCP6004 quad op amp into rows 32-38, columns *e* and *f*, with pin 1 on row 38.  Note that this is the opposite orientation of the EEPROM.

The chips are oriented this way so that only one 3.3V power rail is needed at the top, closest to column *j*.

![Step 1](AssemblyStep1.png)
The NucleoTNC after the first step in the assembly.

### Bypass Capacitors

**Parts Required:**

 * 2 100nF capacitors

Install the 100nF bypass capacitors across the VDD and VSS pins of the two ICs.  The capacitors are going to straddle the chips.  Trim the leads so that the capacitors nearly touch the IC body.

Refer to the datasheets in the [Reference](#Reference) section above to confirm the pin configuration.

 * Insert one capacitor across the EEPROM, inserting one leg into *g21* and the other into *d24*.
 * Insert the other capacitor across the quad op amp, inserting one leg into *d35* and the other into *g35*.

![Step 2](AssemblyStep2.png)
The NucleoTNC with the bypass capacitors installed on the ICs.

### Power and Ground

**Parts Required:**

 * 8 short, stiff wires.

We are now going to connect the power and ground rails to the Nucleo board, the op amp, EEPROM, and the 3.5mm jack which we will be installing shortly.

 * Connect pin *j2* to the upper ground rail (blue).
 * Connect pin *a4* to the lower ground rail.
 * Connect pin *j14* to the upper power rail (red).
 * Connect pin *j21* to the upper power rail.
 * Connect pin *a24* to the lower ground rail.
 * Connect pin *j35* to the upper power rail.
 * Connect pin *a35* to the lower ground rail.
 * Connect pin *a61* to the lower ground rail.

![Step 3](AssemblyStep3.png)
The NucleoTNC with the power and ground rails connected.

### LEDs

**Parts Required:**

 * 2 470 Ohm resistors
 * 1 red LED
 * 1 green LED

For this step we need the two LEDs and two 470 Ohm resistors.  The LEDs will be inserted between the EEPROM and the Nucleo board, so the fit may be a bit tight.  The resistors will be connected from the top power rail to column *i* and the LEDs will straddle the channel in the board.

 * Trim the leads of the two 470 Ohm resistors.
 * Insert one resistor into *j19* and the upper power rail.
 * Insert the other resistor into *j20* and the upper power rail.
 * Trim the leads of the two LEDs, noting their orientation.  The longer lead is the anode and will be oriented towars the top, connecting to the resistor.
 * Insert the red LED into *f19* and *e19*.
 * Insert the green LED into *f20* and *e20*.

Test the LEDs by connecting the Nucleo to USB power and use a jumper wire to connect the cathode of each LED to ground.  Verify that the LED lights up.  When done, disconnect the USB cable from the Nucleo board and remove the jumper wire used in the test.

----
<DIV style="background-color: #ffffce">
Be gentle with the Nucleo board's USB connector.  It can be damaged if too much force is used to insert or extract the USB cable, or if the USB cable is forcefully moved when it is plugged into the device.
</DIV>

![Step 4](AssemblyStep4.png)
The NucleoTNC with the LEDs assembly complete.  The LEDs provide visual indication of transmit and receive.

### 3.5mm Jack

**Parts required:**

 * SparkFun BOB-11570 breakout board.

The sleeve of the 3.5mm jack needs to be connected to ground.  In order to preserve space on the board, the ground connection is going to be made underneath the jack breakout board.

Note: *The breakout board shown in the images is not the same as the one specified in the BOM.*  These layout instructions assume that you are using the SparkFun BOB-11570 in place of the board shown.

 * Insert the 3.5mm jack breakout board into *b58-b61*.
 
![Step 5](AssemblyStep5.png)
The NucleoTNC with the 3.5mm jack installed.  This provides the interface between the TNC and the radio.

### PTT

**Parts required:**

 * 2 2N7000 N-channel MOSFETs
 * 2 220 Ohm resistors.
 * 2 100K Ohm resistors.
 * 1 2.2K Ohm resistor.
 * 5 short, stiff jumper wires.

We will now install the PTT circuitry and connect it to the 3.5mm jack.  The two MOSFETs provide simplex and multiplex PTT signalling.

Refer to the 2N7000 datasheet in the [References](#References) section to understand the pinout and orientation of the TO-92 body.

 * Insert a 100K resistor at *j52* and the upper ground rail.
 * Insert a 100K resistor at *j55* and the upper ground rail.
 * Insert a 2.2K resistor at *j54* and the upper ground rail.
 * Insert a jumper wire across *j51* and the upper ground rail.
 * Insert one 2N7000 with the flat face towards the lower rails at *i51-i53*.
 * Insert one 2N7000 with the flat face towards the lower rails at *i54-i56*.
 * Insert one 220 Ohm resistor across the gap at *e52-f52*.
 * Insert one 220 Ohm resistor across the gap at *e55-f55*.
 * Insert a jumper wire from *g53-g59*.
 * Insert a jumper wire from *h56-h60*.
 * Insert a jumper wire across the gap at *e59-f59*.
 * Insert a jumper wire across the gap at *e60-f60*.

![Step 6](AssemblyStep6.png)
The NucleoTNC with the PTT assembly complete.

### EEPROM Connections

**Parts required:**

 * 4 short, stiff jumper wires.
 * 2 2.2K Ohm resistors

We now need to connect the EEPROM address lines and write-protect line to ground, and connect the I2C SCL and SDA lines to 3.3V through the two pull-up resistors.

Refer to the 24LC32A datasheet in the [References](#References) section to understand the pinout of this chip.

We are going to be connection pins 1,2,3 & 7 to ground. And we are going to connect pins 5 & 6 to the pullup resistors.

 * Insert a jumper wire across *a21* and the lower ground rail.
 * Insert a jumper wire across *a22* and the lower ground rail.
 * Insert a jumper wire across *a23* and the lower ground rail.
 * Insert a jumper wire across *j22* and the upper ground rail.
 * Insert a 2.2k Ohm resistor from *j23* to the upper power rail.
 * Insert a 2.2k Ohm resistor from *j24* to the upper power rail.

![Step 7](AssemblyStep7.png)
The NucleoTNC with the EEPROM assembly complete.

The next two steps are the most involved, as we will be constructing the audio input and output filters.

### Audio Input Circuitry

**Parts required:**

 * 2 BAT85S Schottkey diodes.
 * 1 220nF capacitor.
 * 1 33nF capacitor.
 * 1 1nF capacitor.
 * 1 470pF capactor.
 * 1 10kΩ resistor.
 * 1 16kΩ resistor.
 * 1 30kΩ resistor.
 * 1 510Ω resistor.
 * Various jumper wires.
 
We will now create the audio input section, including the AC coupling cap, DC offset connection, protection diodes, input buffer, and anti-aliasing filter.

 * Connect a wire from *c58-c50*
 * Connect one diode from *a49* to the lower ground rail. The cathode band should be nearest *a49*.
 * Connect one diode from *j49* to the upper power rail.  The cathode bnad should be neares the power rail.
 * Connect a jumper wire across the *e49-f49* gap.
 * Connect the 220nF capacitor from *e50* to *g49*.
 * Connect a wire from *i49* to *i36*.
 * Connect a short jumper from *g37* to *g38*, tying VinA- to VoutA
 * Connect a short jumper from *g32* to *g33*, tying VinB- to VoutB
 * Connect the 470pF capacitor from *j34* to the upper ground rail.
 * Connect the 30kΩ resistor from *j37* to *j41*.
 * Connect the 16kΩ resistor from *g41* to *g34*.  Ensure it is not shorted against the jumper between *g37* and *g38* or the decoupling capacitor at *g35*.
 * Connect the 1nF capacitor from *h41* to *h33*.
 * Connect the 510Ω resistor from *i32* to *i29*.
 * Connect the 33nF capactor from *j29* to the upper ground rail.

![Step 8](AssemblyStep8.png)
The NucleoTNC with the audio input section assembled.

### Audio Output Circuitry

**Parts required:**

 * 1 2.2uF capacitor.
 * 1 33nF capacitor.
 * 1 10nF capacitor.
 * 1 1nF capacitor.
 * 1 470pF capactor.
 * 1 16kΩ resistor.
 * 1 30kΩ resistor.
 * 1 510Ω resistor.
 * 2 470Ω resistors.
 * 1 6.8kΩ resistor.
 * 1 100kΩ resistor.

We will now wire up the audio output circuitry; the reconstruction filter and output buffer.

 * Connect a short jumper from *d32* to *d33*, tying VinC- to VoutC
 * Connect a short jumper from *d37* to *d38*, tying VinD- to VoutD
 * Insert the 30kΩ resistor, spanning *e26* to *e30*.
 * Insert the 16kΩ resistor, spanning *c30* to *c34*.
 * Insert the 470pF capacitor, spanning *a34* to the lower ground rail.
 * Insert the 1nF capacitor, spanning *b30* to *b32*.
 * Insert the 510Ω resistor, spanning *b33* to *b39*.
 * Insert the 33nF capacitor, spanning *a39* to the lower ground rail.
 * Insert the 100kΩ resistor, spanning *d39* to *d36*.
 * Insert the 6.8kΩ resistor, spanning *c36* to *c40*.
 * Insert a small insulated jumper from *a38* to *a41*, being careful to avoid shorting the capacitor at *a39*.
 * Insert a 470Ω resistor from *c41* to *c46*.
 * Insert a 470Ω resistor from *b41* to *b45*.
 * Insert a 10nF capacitor from *a46* to the lower ground rail.
 * Insert a 2.2uF capacitor from *e45* to *e47*.
 * Connect a jumper wire from *d47* to *d60*.
 
![Step 9](AssemblyStep9.png)
The NucleoTNC with the audio output section assembled.  Note that there is a jumper from *d37* to *d38* hidden underneath the 100kΩ resistor.

### Wiring

This is the final step in wiring up the breadboard TNC.  For this step, we are simply using jumper wires to connect the Nucleo board to the various inputs, outputs and control likes of the circuitry we just built.
 
 * Connect a wire from *a12* to *a19* for TX LED control.
 * Connect a wire from *a1* to *a20* for DCD LED control.
 * Connect a wire fram *a5* to *a52* for simplex PTT control.
 * Connect a wire from *a13* to *a55* for multiplex PTT control.
 * Connect a wire from *a14* to *b40* for audio output attenuation control.
 * Connect a wire from *a15* to *i24* for I2C SDA on the EEPROM.
 * Connect a wire from *j6* to *i23* for I2C SCL on the EEPROM.
 * Connect a wire from *j8* to *j46* for DC offest control.
 * Connect a wire from *j9* to *a26* for the DAC audio output.
 * Connect a wire from *j12* to *h29* for the ADC audio input.
 
![Step 10](AssemblyStep10.png)
The completely assembled NucleoTNC.

## Programming

Programming is fairly simple.  When you plug the Nucleo board into your computer, a new drive (mount point) will appear.  Programming is done by simply copying the firmware file into this new mount point.

## Testing & Troubleshooting

After the TNC starts up

## Operation

Before we begin -- a word of warning.

----
<DIV style="background-color: #ffffce">
<span style="font-weight: bold">Do not tranmit RF near the Nucleo TNC.</span>  Use a dummy load when testing and use an external antenna that is mounted well away from the TNC when operating the TNC on the air.  The long wires used in the assembly of the TNC, and the breadboard itself can act as antennas and will conduct RF energy into the TNC components and any equipment you have attached to it, including your computer.  This may cause the TNC to misbehave and can potentially damage your equipment.
</DIV>

----
